In [1]:
# 사전 설치 라이브러리
# pip install gradio, pip install sqlalchemy
# pip install langchain-community, pip install wordcloud, pip install pymysql
import os
import gradio as gr
import glob
from sqlalchemy import create_engine
from config import Config
from document_handler import DocumentHandler
from image_handler import ImageHandler
from chatbot import Chatbot
from feedback_manager import QueryFeedbackManager
from query_generator import QueryGenerator
from natural_language_generator import NaturalLanguageGenerator
from database_interface import DatabaseInterface
from typing import List, Tuple, Optional, Dict, Any
from datetime import datetime
import gc

c:\Python310\aiproject\chatbot_project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 컴포넌트 초기화
db_interface = DatabaseInterface()
query_generator = QueryGenerator()
nl_generator = NaturalLanguageGenerator()
feedback_manager = QueryFeedbackManager(Config.FEEDBACK_FILE)

c:\Python310\aiproject\chatbot_project\query_generator.py:41: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  self.llm = Ollama(model="gemma2", temperature=0)
c:\Python310\aiproject\chatbot_project\query_generator.py:42: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.llm, prompt=self.prompt)


In [3]:
def list_temp_files():
    """temp 폴더 내 파일 목록을 반환하는 함수"""
    if not os.path.exists(Config.TEMP_FOLDER):
        os.makedirs(Config.TEMP_FOLDER)
    return [os.path.basename(f) for f in glob.glob(os.path.join(Config.TEMP_FOLDER, "*.txt"))]

In [4]:
def process_question(question: str) -> Tuple[str, str, str]:
        """사용자 질문을 처리하고 결과를 반환합니다."""
        feedback_manager.last_question = question
        
        # 유사한 피드백 찾기
        similar_feedback = feedback_manager.find_similar_feedback(question)
        feedback_info = f"이전 피드백: {similar_feedback['feedback']}" if similar_feedback else ""
        
        # 쿼리 생성 및 실행
        schema_info = db_interface.get_schema_info()
        query = query_generator.generate_query(question, schema_info, feedback_info)
        columns, results = db_interface.execute_query(query)
        
        # 결과 생성
        if columns is None:
            return "쿼리 실행 실패", "오류가 발생했습니다.", query
        
        result_text = "\n".join([str(row) for row in results])
        natural_explanation = nl_generator.generate_summary(columns, results)
        
        return result_text, natural_explanation, query

In [5]:
def handle_feedback(feedback: str) -> str:
        """피드백을 처리하고 저장합니다."""
        if not feedback_manager.last_question:
            return "피드백을 저장할 질문이 없습니다."
        
        feedback_data = feedback_manager.load_feedback()
        feedback_data.append({
            "id": len(feedback_data) + 1,
            "question": feedback_manager.last_question,
            "feedback": feedback,
            "timestamp": datetime.now().isoformat()
        })
        feedback_manager.save_feedback(feedback_data)
        return "피드백이 성공적으로 저장되었습니다. 다음 검색에 반영됩니다."

In [6]:
document_handler = DocumentHandler()
image_handler = ImageHandler()
chatbot = Chatbot(document_handler, image_handler)

c:\Python310\aiproject\chatbot_project\document_handler.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings_model = HuggingFaceEmbeddings(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
c:\Python310\aiproject\chatbot_project\chatbot.py:22: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -

In [7]:
with gr.Blocks(theme=gr.themes.Soft(), css="footer {display: none !important}") as iface:
    gr.Markdown("# 정보 제공 AI 챗봇")

    with gr.Tabs():
        # 채팅 탭
        with gr.TabItem("채팅"):
            chatbot_interface = gr.Chatbot(height=600)
            msg = gr.Textbox(label="질문을 입력하세요", lines=1)
            image_input = gr.Image(type="pil", label="이미지 업로드 (선택사항)")

            with gr.Row():
                submit_btn = gr.Button("전송", variant="primary")
                clear_btn = gr.Button("대화 내용 지우기")

            gr.Examples(
                examples=[
                    ["휴먼(주)의 주소는 어디인가요?", None],
                    ["휴먼(주)가 보유한 솔루션은 무엇인가요?", None],
                    ["휴먼(주)의 영업대표는 누구인가요?", None]
                ],
                inputs=[msg, image_input]
            )

            msg.submit(chatbot.chat, [msg, chatbot_interface, image_input], 
                     [msg, chatbot_interface, image_input])
            submit_btn.click(chatbot.chat, [msg, chatbot_interface, image_input], 
                           [msg, chatbot_interface, image_input])
            clear_btn.click(lambda: (None, None), None, [chatbot_interface, image_input])

        # 정보 추가 탭
        with gr.TabItem("정보 추가"):
            new_info_input = gr.Textbox(label="새로운 정보 추가", lines=3)
            add_info_btn = gr.Button("정보 추가하기")
            info_status = gr.Textbox(label="상태 메시지", interactive=False)

            add_info_btn.click(document_handler.add_new_information, 
                             [new_info_input], 
                             [info_status])

        # 파일 관리 탭
        with gr.TabItem("파일 관리"):
            gr.Markdown("## temp 폴더에 파일 추가 및 질문")

            file_input = gr.File(label="Windows에서 파일 추가")
            add_file_btn = gr.Button("파일 추가하기")
            file_status = gr.Textbox(label="상태 메시지", interactive=False)

            file_list = gr.Dropdown(
                label="temp 폴더의 파일 선택",
                choices=list_temp_files(),
                multiselect=False,
                interactive=True
            )

            def update_file_list(file):
                status_message, _ = document_handler.add_new_file(file)
                updated_files = list_temp_files()
                return status_message, gr.update(choices=updated_files)

            add_file_btn.click(
                update_file_list,
                inputs=[file_input],
                outputs=[file_status, file_list]
            )

            question_input = gr.Textbox(label="질문 입력", lines=2, 
                                      placeholder="질문을 입력하세요.")
            ask_btn = gr.Button("질문 전송", variant="primary")
            answer_box = gr.Textbox(label="답변", interactive=False)

            def ask_from_selected_file(selected_file, question):
                if not selected_file:
                    return "먼저 파일을 선택해주세요."
                if not question.strip():
                    return "질문을 입력해주세요."

                try:
                    file_path = os.path.join(Config.TEMP_FOLDER, selected_file)
                    if not os.path.exists(file_path):
                        return f"선택된 파일을 찾을 수 없습니다: {selected_file}"

                    custom_db = document_handler.initialize_db(file_path, use_main_only=False)
                    temp_qa_chain = chatbot.initialize_qa_chain(custom_db)

                    response = temp_qa_chain({"question": question, "chat_history": []})

                    answer = response["answer"]
                    if 'source_documents' in response:
                        sources = set([os.path.basename(doc.metadata.get('source', 'Unknown')) 
                                     for doc in response['source_documents']])
                        source_info = f"\n\n참고 파일: {', '.join(sources)}" if sources else ""
                        answer += source_info

                    return answer

                except Exception as e:
                    return f"오류가 발생했습니다: {str(e)}"

            ask_btn.click(
                ask_from_selected_file,
                inputs=[file_list, question_input],
                outputs=answer_box
            )

        # 전체 내용 보기 탭
        with gr.TabItem("전체 내용 보기"):
            view_content_btn = gr.Button("전체 내용 보기")
            content_display = gr.Textbox(label="전체 내용", lines=10)

            def view_all_content():
                try:
                    with open(Config.MAIN_DOC_PATH, 'r', encoding='utf-8') as f:
                        content = "=== 메인 문서 내용 ===\n" + f.read().strip() + "\n\n"

                    if os.path.exists(Config.TEMP_FOLDER):
                        for filename in os.listdir(Config.TEMP_FOLDER):
                            if filename.endswith('.txt'):
                                file_path = os.path.join(Config.TEMP_FOLDER, filename)
                                try:
                                    with open(file_path, 'r', encoding='utf-8') as f:
                                        content += f"=== {filename} ===\n{f.read().strip()}\n\n"
                                except Exception as e:
                                    content += f"=== {filename} ===\n(읽기 오류: {str(e)})\n\n"
                    else:
                        content += "temp 폴더가 존재하지 않습니다.\n\n"

                    return content
                except Exception as e:
                    return f"전체 내용 보기 중 오류 발생: {str(e)}"

            view_content_btn.click(view_all_content, None, content_display)

        # 단어시각화 탭
        with gr.TabItem("단어시각화"):
            generate_btn = gr.Button("Word Cloud 생성")
            output_image = gr.Image()
            generate_btn.click(
                lambda: image_handler.generate_wordcloud(), 
                None, 
                output_image
            )
            
        # DB연결 조회 탭
        with gr.TabItem("DB연결 조회"):
            question_input = gr.Textbox(
                label="질문 입력",
                placeholder="데이터베이스 관련 질문을 입력하세요.",
                lines=2
            )
            submit_button = gr.Button("질문 제출")
            query_area = gr.Textbox(label="생성된 쿼리", lines=4, interactive=False)
            result_area = gr.Textbox(label="쿼리 결과", lines=4, interactive=False)
            explanation_area = gr.Textbox(label="자연어 설명", lines=6, interactive=False)

            submit_button.click(
                process_question,
                inputs=question_input,
                outputs=[result_area, explanation_area, query_area]
            )

        # 피드백 제공 탭
        with gr.TabItem("피드백 제공"):
            feedback_input = gr.Textbox(label="피드백", lines=4)
            feedback_button = gr.Button("피드백 제출")
            feedback_output = gr.Textbox(label="상태", interactive=False)

            feedback_button.click(
                handle_feedback,
                inputs=feedback_input,
                outputs=feedback_output
            )

C:\Users\yj\AppData\Local\Temp\ipykernel_10848\1506479090.py:7: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_interface = gr.Chatbot(height=600)


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [8]:
iface.launch(
        server_port=7860,
        server_name="127.0.0.1",
        inbrowser=False,  # 자체 브라우저 실행은 비활성화
        show_error=True,  # 오류 표시 활성화
        quiet=False,       # 로그 출력 활성화
        prevent_thread_lock=True  # 스레드 블로킹 방지
    )

INFO:httpx:HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [11]:
iface.close()

Closing server running on port: 7860


In [10]:
# Garbage Collection 실행
gc.collect()  # 메모리 정리

16977

c:\Python310\aiproject\chatbot_project\query_generator.py:46: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.chain.run(
C:\Users\yj\AppData\Local\Temp\ipykernel_10848\1506479090.py:85: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = temp_qa_chain({"question": question, "chat_history": []})




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    당신은 휴먼(주) 정보를 제공하는 AI 어시스턴트입니다. 모든 답변은 한국어로 답변해 주세요.

    아래는 이전 대화 내용입니다:
    

    관련 문서 내용:
    한국의 역사는 수천 년에 걸쳐 이어져 온 긴 여정 속에서 다양한 문화와 전통이 형성되고 발전해 왔습니다. 고조선에서 시작해 삼국 시대의 경쟁, 그리고 통일 신라와 고려를 거쳐 조선까지, 한반도는 많은 변화를 겪었습니다.

고조선은 기원전 2333년 단군왕검에 의해 세워졌다고 전해집니다. 이는 한국 역사상 최초의 국가로, 한민족의 시원이라 할 수 있습니다. 이후 기원전 1세기경에는 한반도와 만주 일대에서 여러 소국이 성장하며 삼한 시대로 접어듭니다.

4세기경, 고구려, 백제, 신라의 삼국이 한반도의 주요 세력으로 부상했습니다. 이 시기는 삼국이 각각 문화와 기술, 무력을 발전시키며 경쟁적으로 성장한 시기로, 한국 역사에서 중요한 전환점을 마련했습니다. 특히 고구려는 북방의 강대국으로 성장하여 중국과도 여러 차례 전쟁을 벌였습니다.

7세기 말, 신라는 당나라와 연합하여 백제와 고구려를 차례로 정복하고, 한반도 최초의 통일 국가인 통일 신라를 건립합니다. 이 시기에 신라는 불교를 국교로 채택하며 문화와 예술이 크게 발전했습니다.

그러나 10세기에 이르러 신라는 내부의 분열과 외부의 압력으로 쇠퇴하고, 이를 대체하여 고려가 성립됩니다. 고려 시대에는 과거제도의 도입과 더불어 청자 등 고려 고유의 문화가 꽃피었습니다.

조선은 1392년 이성계에 의해 건국되어, 1910년까지 이어졌습니다. 조선 초기에는 세종대왕이 한글을 창제하여 백성들의 문해율을 높이는 등 문화적, 과학적 성취가 이루어졌습니다. 그러나 조선 후기에는 내부적으로 실학의 발전과 함께 사회적 변화가 모색되었으나, 외부로부터의 압력은


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    당신은 휴먼(주) 정보를 제공하는 AI 어시스턴트입니다. 모든 답변은 한국어로 답변해 주세요.

    아래는 이전 대화 내용입니다:
    
Human: 중심내용은 무엇입니까?
Assistant: 제공된 문서는 휴먼(주)에 대한 정보와 한국 역사 개요를 담고 있습니다. 따라서 중심 내용은 **휴먼(주) 회사 소개 및 한국 역사** 입니다. 




    관련 문서 내용:
    휴먼(주) 대표이사는? 홍길동 대표입니다.
휴먼 주요 솔루션은? ERP(전사적자원관리) 시스템, 업무포털, PMS 등이 있습니다.
휴먼 직원은 몇 명인가요? 40명 이내입니다.
휴먼 영업대표는? 강건한 과장입니다.
휴먼 경영지원팀장은? 김샛빛 차장입니다.
휴먼 미션은? 기술과 비즈니스를 융합하여 새로운 미래를 만드는 기업
휴먼 비전은? 정보기술의 가치 창출로 고객에겐 가치를 직원에겐 행복을 주는 기업
휴먼 인재상은? 성실, 선의(선한 의도), 능력, 실적
휴먼 주소는? 수원시 팔달구 중부대로 100 
휴먼 분위기는? 일할 때는 조용합니다. 존중하는 분위기.
휴먼 단체 해외여행 간 곳은? 중국 상하이, 항주, 심천, 홍콩, 마카오
조용한 이사 성격은? 말이 없고 조용합니다. 그래서 재미가 없지만 책임감이 있음.
휴먼 2024년 영업 매출 목표액은? 매출액 목표는 30억입니다.
휴먼에서 가까운 역은? 수원역
휴먼 창립년도는? 2010년 1월 1일
휴먼 홈페이지 주소는? http://www.human.co.kr
휴먼 주요사업은? SI(시스템통합), SM(시스템유지보수), 솔루션개발, ISP(정보화전략계획), ISMP(정보화전략마스터플랜), AI(인공지능), 시각화
성진실 차장은 대전에서 살고 있고, 2024년 프로젝트로 인하여 서울로 출퇴근 하고 있음. 그래서